In [2]:
import boto3

region = 'us-east-1'

s3_client = boto3.client('s3', region_name=region)

# 👇️ print region name
print(s3_client.meta.region_name)

print(s3_client)

us-east-1


In [3]:
import boto3

region = 'ap-south-1'

s3_client = boto3.client('s3')

print(s3_client.meta.region_name)

print(s3_client)

us-east-1


In [4]:
import boto3

s3_client = boto3.client('s3')

print(s3_client.meta.region_name)

print(s3_client)

us-east-1


In [5]:
import os
import boto3

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

s3_client = boto3.client('s3')

print(s3_client.meta.region_name) # 👉️ us-east-1

print(s3_client)

us-east-1


In [6]:
import os
import boto3


def main(event, context):
    lambda_region = os.environ['AWS_REGION']

    print(f'Lambda function region {lambda_region}')

    s3_client = boto3.client('s3', region_name=lambda_region)

In [7]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket= 'mybucketsagemakerr'
print("Using bucket" + bucket)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Yash\AppData\Local\sagemaker\sagemaker\config.yaml
Using bucketmybucketsagemakerr


In [8]:
import pandas as pd

In [9]:

df=pd.read_csv(r'C:/Users/Yash/Downloads/cars-database.csv')



In [10]:
df.head()

,mpg,cylinders,cubicinches,hp,weightlbs,time-to-60,year,brand
0,14.0,8,350,165,4209,12,1972,US.
1,31.9,4,89,71,1925,14,1980,Europe.
2,17.0,8,302,140,3449,11,1971,US.
3,15.0,8,400,150,3761,10,1971,US.
4,30.5,4,98,63,2051,17,1978,US.


In [11]:
df.shape

(261, 8)

In [12]:
df['mpg'].value_counts(normalize=True)

mpg
14.0    0.061303
18.0    0.053640
13.0    0.049808
16.0    0.045977
26.0    0.038314
          ...   
26.5    0.003831
30.9    0.003831
29.9    0.003831
26.8    0.003831
23.5    0.003831
Name: proportion, Length: 103, dtype: float64

In [13]:
df.columns

Index(['mpg', 'cylinders', 'cubicinches', 'hp', 'weightlbs', 'time-to-60',
       'year', 'brand'],
      dtype='object')

In [14]:
df.isnull().mean() * 100

mpg            0.0
cylinders      0.0
cubicinches    0.0
hp             0.0
weightlbs      0.0
time-to-60     0.0
year           0.0
brand          0.0
dtype: float64

In [15]:
features= list(df.columns)
features

['mpg',
 'cylinders',
 'cubicinches',
 'hp',
 'weightlbs',
 'time-to-60',
 'year',
 'brand']

In [16]:
label = features.pop(-1)
label

'brand'

In [17]:
x=df[features]
y=df[label]

In [18]:
x.head()

,mpg,cylinders,cubicinches,hp,weightlbs,time-to-60,year
0,14.0,8,350,165,4209,12,1972
1,31.9,4,89,71,1925,14,1980
2,17.0,8,302,140,3449,11,1971
3,15.0,8,400,150,3761,10,1971
4,30.5,4,98,63,2051,17,1978


In [19]:
y.head()

0         US.
1     Europe.
2         US.
3         US.
4         US.
Name: brand, dtype: object

In [20]:
x.shape

(261, 7)

In [21]:
y.value_counts()

brand
 US.        162
 Japan.      51
 Europe.     48
Name: count, dtype: int64

In [22]:
X_train, X_test, y_train, y_test = train_test_split( x , y , test_size = 0.15 , random_state = 0 )

In [23]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(221, 7)
(40, 7)
(221,)
(40,)


In [24]:
import pandas as pd

In [25]:
trainX = pd.DataFrame(X_train)
trainX[label]= y_train


testX = pd.DataFrame(X_test)
testX[label]= y_train

In [26]:
print(trainX.shape)
print(testX.shape)

(221, 8)
(40, 8)


In [27]:
trainX.head()

,mpg,cylinders,cubicinches,hp,weightlbs,time-to-60,year,brand
5,23.0,8,350,125,3900,17,1980,US.
22,24.0,6,200,81,3012,18,1977,US.
181,33.8,4,97,67,2145,18,1981,Japan.
118,34.4,4,98,65,2045,16,1982,US.
12,16.0,8,302,140,4141,14,1975,US.


In [28]:
trainX.isnull().sum()

mpg            0
cylinders      0
cubicinches    0
hp             0
weightlbs      0
time-to-60     0
year           0
brand          0
dtype: int64

In [29]:
trainX.to_csv("train-V-1.csv",index = False)
testX.to_csv("test-V-1.csv",index = False)

In [30]:
bucket

'mybucketsagemakerr'

In [31]:
sk_prefix = "sagemaker/cars-database/sklearncontainer"
trainpath = sess.upload_data(
    path= "train-V-1.csv", bucket= bucket, key_prefix= sk_prefix
)
testpath = sess.upload_data(
    path= "test-V-1.csv", bucket= bucket, key_prefix= sk_prefix
)
print(trainpath)
print(testpath)


s3://mybucketsagemakerr/sagemaker/cars-database/sklearncontainer/train-V-1.csv
s3://mybucketsagemakerr/sagemaker/cars-database/sklearncontainer/test-V-1.csv


In [32]:
bucket

'mybucketsagemakerr'

In [33]:
%%writefile script.py

import argparse
import joblib
import os

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor


# inference functions ---------------
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf


if __name__ == "__main__":
    print("extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # to simplify the demo we don't use all sklearn RandomForest hyperparameters
    parser.add_argument("--n-estimators", type=int, default=10)
    parser.add_argument("--min-samples-leaf", type=int, default=3)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")
    parser.add_argument(
        "--features", type=str
    )  # in this script we ask user to explicitly name features
    parser.add_argument(
        "--target", type=str
    )  # in this script we ask user to explicitly name the target

    args, _ = parser.parse_known_args()

    print("reading data")
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    print("building training and testing datasets")
    X_train = train_df[args.features.split()]
    X_test = test_df[args.features.split()]
    y_train = train_df[args.target]
    y_test = test_df[args.target]

    # train
    print("training model")
    model = RandomForestRegressor(
        n_estimators=args.n_estimators, min_samples_leaf=args.min_samples_leaf, n_jobs=-1
    )

    model.fit(X_train, y_train)

    # print abs error
    print("validating model")
    abs_err = np.abs(model.predict(X_test) - y_test)

    # print couple perf metrics
    for q in [10, 50, 90]:
        print("AE-at-" + str(q) + "th-percentile: " + str(np.percentile(a=abs_err, q=q)))

    # persist model
    path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, path)
    print("model persisted at " + path)
    print(args.min_samples_leaf)

Overwriting script.py


In [34]:
# We use the Estimator from the SageMaker Python SDK
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::576629149578:user/yashworkspace",
    instance_count=1,
    instance_type="ml.c5.xlarge",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="rf-scikit",
    metric_definitions=[{"Name": "median-AE", "Regex": "AE-at-50th-percentile: ([0-9.]+).*$"}],
    hyperparameters={
        "n-estimators": 100,
        "min-samples-leaf": 3,
        "features": "MedInc HouseAge AveRooms AveBedrms Population AveOccup Latitude Longitude",
        "target": "target",
    },
)

In [36]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Principal': {'Service': 'sagemaker.amazonaws.com'},
   'Action': 'sts:AssumeRole'}]}

In [39]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:DeleteObject",
                "s3:AbortMultipartUpload"
            ],
            "Resource": [
                "arn:aws:s3:::DOC-EXAMPLE-BUCKET1/*",
                "arn:aws:s3:::DOC-EXAMPLE-BUCKET2/*"
            ]
        }, 
        {
            "Effect": "Allow",
            "Action": [
                "s3:CreateBucket",
                "s3:GetBucketLocation",
                "s3:ListBucket",
                "s3:ListAllMyBuckets",
                "s3:GetBucketCors",
                "s3:PutBucketCors"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetBucketAcl",
                "s3:PutObjectAcl"
            ],
            "Resource": [
                "arn:aws:s3:::DOC-EXAMPLE-BUCKET1",
                "arn:aws:s3:::DOC-EXAMPLE-BUCKET2"
            ]
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Action': ['s3:GetObject',
    's3:PutObject',
    's3:DeleteObject',
    's3:AbortMultipartUpload'],
   'Resource': ['arn:aws:s3:::DOC-EXAMPLE-BUCKET1/*',
    'arn:aws:s3:::DOC-EXAMPLE-BUCKET2/*']},
  {'Effect': 'Allow',
   'Action': ['s3:CreateBucket',
    's3:GetBucketLocation',
    's3:ListBucket',
    's3:ListAllMyBuckets',
    's3:GetBucketCors',
    's3:PutBucketCors'],
   'Resource': '*'},
  {'Effect': 'Allow',
   'Action': ['s3:GetBucketAcl', 's3:PutObjectAcl'],
   'Resource': ['arn:aws:s3:::DOC-EXAMPLE-BUCKET1',
    'arn:aws:s3:::DOC-EXAMPLE-BUCKET2']}]}

In [41]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Principal': {'Service': 'sagemaker.amazonaws.com'},
   'Action': 'sts:AssumeRole'}]}

In [42]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "iam:PassRole"
            ],
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "iam:PassedToService": "sagemaker.amazonaws.com"
                }
            }
        },
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:DescribeEndpointConfig",
                "sagemaker:DescribeModel",
                "sagemaker:InvokeEndpoint",
                "sagemaker:ListTags",
                "sagemaker:DescribeEndpoint",
                "sagemaker:CreateModel",
                "sagemaker:CreateEndpointConfig",
                "sagemaker:CreateEndpoint",
                "sagemaker:DeleteModel",
                "sagemaker:DeleteEndpointConfig",
                "sagemaker:DeleteEndpoint",
                "cloudwatch:PutMetricData",
                "logs:CreateLogStream",
                "logs:PutLogEvents",
                "logs:CreateLogGroup",
                "logs:DescribeLogStreams",
                "s3:GetObject",
                "s3:PutObject",
                "s3:ListBucket",
                "ecr:GetAuthorizationToken",
                "ecr:BatchCheckLayerAvailability",
                "ecr:GetDownloadUrlForLayer",
                "ecr:BatchGetImage"
            ],
            "Resource": "*"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Action': ['iam:PassRole'],
   'Resource': '*',
   'Condition': {'StringEquals': {'iam:PassedToService': 'sagemaker.amazonaws.com'}}},
  {'Effect': 'Allow',
   'Action': ['sagemaker:DescribeEndpointConfig',
    'sagemaker:DescribeModel',
    'sagemaker:InvokeEndpoint',
    'sagemaker:ListTags',
    'sagemaker:DescribeEndpoint',
    'sagemaker:CreateModel',
    'sagemaker:CreateEndpointConfig',
    'sagemaker:CreateEndpoint',
    'sagemaker:DeleteModel',
    'sagemaker:DeleteEndpointConfig',
    'sagemaker:DeleteEndpoint',
    'cloudwatch:PutMetricData',
    'logs:CreateLogStream',
    'logs:PutLogEvents',
    'logs:CreateLogGroup',
    'logs:DescribeLogStreams',
    's3:GetObject',
    's3:PutObject',
    's3:ListBucket',
    'ecr:GetAuthorizationToken',
    'ecr:BatchCheckLayerAvailability',
    'ecr:GetDownloadUrlForLayer',
    'ecr:BatchGetImage'],
   'Resource': '*'}]}

In [43]:
{
    "Effect": "Allow",
    "Action": [
        "ec2:CreateNetworkInterface",
        "ec2:CreateNetworkInterfacePermission",
        "ec2:DeleteNetworkInterface",
        "ec2:DeleteNetworkInterfacePermission",
        "ec2:DescribeNetworkInterfaces",
        "ec2:DescribeVpcs",
        "ec2:DescribeDhcpOptions",
        "ec2:DescribeSubnets",
        "ec2:DescribeSecurityGroups"
    ]
}

{'Effect': 'Allow',
 'Action': ['ec2:CreateNetworkInterface',
  'ec2:CreateNetworkInterfacePermission',
  'ec2:DeleteNetworkInterface',
  'ec2:DeleteNetworkInterfacePermission',
  'ec2:DescribeNetworkInterfaces',
  'ec2:DescribeVpcs',
  'ec2:DescribeDhcpOptions',
  'ec2:DescribeSubnets',
  'ec2:DescribeSecurityGroups']}

In [45]:
! python script.py --n-estimators 100 \
                   --min-samples-leaf 2 \
                   --model-dir ./ \
                   --train ./ \
                   --test ./ \
                   --features 'MedInc HouseAge AveRooms AveBedrms Population AveOccup Latitude Longitude' \
                   --target target

extracting arguments

Traceback (most recent call last):
  File "c:\Users\Yash\myenv\conda-meta\script.py", line 46, in <module>
    X_train = train_df[args.features.split()]
              ~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Yash\anaconda3\Lib\site-packages\pandas\core\frame.py", line 3899, in __getitem__
    indexer = self.columns._get_indexer_strict(key, "columns")[1]
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Yash\anaconda3\Lib\site-packages\pandas\core\indexes\base.py", line 6115, in _get_indexer_strict
    self._raise_if_missing(keyarr, indexer, axis_name)
  File "c:\Users\Yash\anaconda3\Lib\site-packages\pandas\core\indexes\base.py", line 6176, in _raise_if_missing
    raise KeyError(f"None of [{key}] are in the [{axis_name}]")
KeyError: "None of [Index([''MedInc'], dtype='object')] are in the [columns]"



reading data
building training and testing datasets


In [62]:
# We use the Estimator from the SageMaker Python SDK
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="role",
    instance_count=1,
    instance_type="ml.c5.xlarge",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="rf-scikit",
    metric_definitions=[{"Name": "median-AE", "Regex": "AE-at-50th-percentile: ([0-9.]+).*$"}],
    hyperparameters={
        "n-estimators": 100,
        "min-samples-leaf": 0,
        "features": "MedInc HouseAge AveRooms AveBedrms Population AveOccup Latitude Longitude",
        "target": "target",
    },
)

In [63]:
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)

NoSuchEntityException: An error occurred (NoSuchEntity) when calling the GetRole operation: The role with name role cannot be found.

In [52]:
source = "source.tar.gz"
project = "scikitlearn-train-from-boto3"

tar = tarfile.open(source, "w:gz")
tar.add("script.py")
tar.close()

s3 = boto3.client("s3")
s3.upload_file(source, bucket, project + "/" + source)

NameError: name 'tarfile' is not defined

In [60]:
import boto3

In [61]:
from sagemaker.sklearn.estimator import SKLearn

sklearn_estimator = SKLearn(
    entry_point='script.py',
    role = "arn:aws:iam::576629149578:user/yashworkspace",
    instance_count=1,
    instance_type='ml.m4.xlarge',
    framework_version='0.20.0',
    base_job_name='rf-scikit')

sklearn_estimator.fit({'train':trainpath, 'test': testpath}, wait=False)

sklearn_estimator.latest_training_job.wait(logs='None')
artifact = m_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name)['ModelArtifacts']['S3ModelArtifacts']

print('Model artifact persisted at ' + artifact)

INFO:sagemaker:Creating training-job with name: rf-scikit-2024-01-11-16-45-23-886


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: 1 validation error detected: Value 'arn:aws:iam::576629149578:user/yashworkspace' at 'roleArn' failed to satisfy constraint: Member must satisfy regular expression pattern: ^arn:aws[a-z\-]*:iam::\d{12}:role/?[a-zA-Z_0-9+=,.@\-_/]+$

In [57]:
from sagemaker.sklearn.estimator import SKLearn
sklearn_estimator = SKLearn(
    entry_point='script.py',
    role = "arn:aws:iam::576629149578:user/yashworkspace",
    instance_count=1,
    instance_type='ml.m4.xlarge',
    framework_version='0.20.0',
    base_job_name='rf-scikit')

In [58]:
sklearn_estimator.fit({'train':trainpath, 'test': testpath}, wait=False)

INFO:sagemaker:Creating training-job with name: rf-scikit-2024-01-11-16-28-55-252


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: 1 validation error detected: Value 'arn:aws:iam::576629149578:user/yashworkspace' at 'roleArn' failed to satisfy constraint: Member must satisfy regular expression pattern: ^arn:aws[a-z\-]*:iam::\d{12}:role/?[a-zA-Z_0-9+=,.@\-_/]+$

In [59]:
sklearn_estimator.latest_training_job.wait(logs='None')
artifact = m_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name)['ModelArtifacts']['S3ModelArtifacts']

print('Model artifact persisted at ' + artifact)

AttributeError: 'NoneType' object has no attribute 'wait'